In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
X, y = fetch_openml("adult",version = 2,  return_X_y=True)
X["workclass"] = X["workclass"].fillna(X["workclass"].mode()[0])
X["occupation"] = X["occupation"].fillna(X["occupation"].mode()[0])
X["native-country"] = X["native-country"].fillna(X["native-country"].mode()[0])
label_encoder = LabelBinarizer()
feature_encoder = OneHotEncoder(sparse_output=False)
poly = PolynomialFeatures(degree=3)
scaler = StandardScaler()
y = label_encoder.fit_transform(y)
cat_features = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
num_features = ["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]
X[num_features] = scaler.fit_transform(X[num_features])
df_poly = pd.DataFrame(poly.fit_transform(X[num_features]))
df_encoded = pd.DataFrame(feature_encoder.fit_transform(X[cat_features]))
X.drop(X[cat_features], inplace=True, axis="columns")
X = pd.concat([X, df_encoded, df_poly], axis="columns")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X.columns = X.columns.astype(str)
y = y.squeeze()
x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size = 0.1)
reg = LogisticRegression()
reg.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(reg.predict(x_test), y_test)

0.8542476970317298

In [ ]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml("adult",version = 2,  return_X_y=True)
X["workclass"] = X["workclass"].fillna(X["workclass"].mode()[0])
X["occupation"] = X["occupation"].fillna(X["occupation"].mode()[0])
X["native-country"] = X["native-country"].fillna(X["native-country"].mode()[0])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)
pre_pipeline = Pipeline([
    ("normalizer", StandardScaler()),
    ("poly", PolynomialFeatures())
])
column_transformer = ColumnTransformer([
    ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ("num_poly", pre_pipeline, num_features)
])

pipeline = Pipeline([
    ("pre" , column_transformer),
    ("clf" , LogisticRegression(solver = "lbfgs", max_iter=1000))
])
param_grid = {
    "pre__num_poly__poly__degree" : [3, 4, 5],
    "clf__C" : [0.1, 0.01, 1],
}
grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=2, scoring="accuracy", n_jobs=-1)

In [ ]:
# grid.fit(x_train, y_train)

KeyboardInterrupt: 

In [ ]:
accuracy_score(grid.best_estimator_.predict(x_test), y_test)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:
pd.DataFrame(grid.cv_results_).sort_values("rank_test_score")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, stratify=y)

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset, random_split
X, y = fetch_openml("adult",version = 2,  return_X_y=True)
X["workclass"] = X["workclass"].fillna(X["workclass"].mode()[0])
X["occupation"] = X["occupation"].fillna(X["occupation"].mode()[0])
X["native-country"] = X["native-country"].fillna(X["native-country"].mode()[0])
label_encoder = LabelBinarizer()
feature_encoder = OneHotEncoder(sparse_output=False)
poly = PolynomialFeatures(degree=3)
scaler = StandardScaler()
y = label_encoder.fit_transform(y)
cat_features = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
num_features = ["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]
X[num_features] = scaler.fit_transform(X[num_features])
df_poly = pd.DataFrame(poly.fit_transform(X[num_features]))
df_encoded = pd.DataFrame(feature_encoder.fit_transform(X[cat_features]))
X.drop(X[cat_features], inplace=True, axis="columns")
X = pd.concat([X, df_encoded, df_poly], axis="columns")
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)
train_size = int(len(x_train) * 0.95)
val_size = (len(x_train) - train_size)
data = TensorDataset(x_train_tensor, y_train_tensor)
train_data , val_data = random_split(data, [train_size, val_size])
train_loader = DataLoader(train_data, batch_size = 32)
val_loader = DataLoader(val_data, batch_size = 32)
class MyArtificialNeuralNetWork(nn.Module):
  def __init__(self, n_features, n_classes):
    super().__init__()
    self.sequential = nn.Sequential(
        nn.Linear(n_features, 128),
        nn.Dropout(0.25),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.Dropout(0.25),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Linear(64, 64),
        nn.Dropout(0.25),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Linear(64, n_classes)
    )
  def forward(self, x):
    return self.sequential(X)
from tqdm import trange
import matplotlib.pyplot as plt
list_train_loss = []
list_val_loss = []
model = MyArtificialNeuralNetWork(189, 2)
loss_function = nn.BCEWithLogitsLoss()
optimizer = Adam(params=model.parameters(), lr=1e-4, weight_decay=1e-5)
for epoch in (pbar := trange(10)):
  for x_batch, y_batch in train_loader:
    model.train()
    y_pred = model(x_batch)
    train_loss = loss_function(y_pred, y_batch.squeeze())
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    list_train_loss.append(train_loss.detach())
  model.eval()
  with torch.inference_mode():
    for x_val, y_val in val_loader:
      val_loss = loss_function(model(x_val), y_val)
      list_val_loss.append(val_loss)
plt.plot(range(len(list_train_loss)), list_train_loss, c = "r")
plt.plot(range(len(list_val_loss)), list_val_loss, c = "g")

  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)
train_size = int(len(x_train_tensor) * 0.95)
val_size = (len(x_train_tensor) - train_size)
X_data = TensorDataset(x_train_tensor, y_train_tensor)
X_train, X_val = random_split(X_data, [train_size, val_size])
X_train_loader = DataLoader(X_train, batch_size=32, shuffle=True)
X_val_loader = DataLoader(X_val, batch_size=32, shuffle=True)

In [ ]:
class MyArtificialNeuralNetWork(nn.Module):
  def __init__(self, n_features, n_classes):
    super().__init__()
    self.sequential = nn.Sequential(
        nn.Linear(n_features, 128),
        nn.Dropout(0.25),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.Dropout(0.25),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Linear(64, 64),
        nn.Dropout(0.25),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Linear(64, n_classes)
    )
  def forward(self, x):
    return self.sequential(X)

In [ ]:
from tqdm import trange
import matplotlib.pyplot as plt
list_train_loss = []
list_val_loss = []
model = MyArtificialNeuralNetWork(189, 2)
loss_function = nn.BCEWithLogitsLoss()
optimizer = Adam(params=model.parameters(), lr=1e-4, weight_decay=1e-5)
for epoch in (pbar := trange(10)):
  for x_batch, y_batch in train_loader:
    model.train()
    y_pred = model(x_batch)
    train_loss = loss_function(y_pred, y_batch.squeeze())
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    list_train_loss.append(train_loss.detach())
  model.eval()
  with torch.inference_mode():
    for x_val, y_val in val_loader:
      val_loss = loss_function(model(x_val), y_val)
      list_val_loss.append(val_loss)
plt.plot(range(len(list_train_loss)), list_train_loss, c = "r")
plt.plot(range(len(list_val_loss)), list_val_loss, c = "g")

  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame

In [ ]:
# %reset -f